In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # 경고문 무시

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.naive_bayes import GaussianNB


In [2]:
# 데이터 불러오기
from google.colab import drive
drive.mount('/content/drive')
train_src = '/content/drive/MyDrive/Colab Notebooks/패턴인식/train.csv'

df = pd.read_csv(train_src)

X = df.drop(['id', 'shares', 'y'], axis=1)
y = df['y']

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Mounted at /content/drive


In [3]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = ['data_channel', 'weekday']

# 전처리 파이프라인
numeric_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(drop='first', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_pipe, num_cols),
    ('cat', categorical_pipe, cat_cols)
])

In [5]:
# GaussianNB
pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', GaussianNB())
])

In [6]:
# 5-Fold CV (only on train_val)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = ['accuracy', 'f1', 'roc_auc']

cv_results = cross_validate(pipe, X_trainval, y_trainval, cv=cv, scoring=scoring)

acc = cv_results['test_accuracy']
f1 = cv_results['test_f1']
auc = cv_results['test_roc_auc']
comp = (acc + f1 + auc) / 3

print("5-Fold CV (only on train_val)")
for i in range(len(acc)):
    print(f"[Fold {i+1}] Accuracy: {acc[i]:.4f}, F1: {f1[i]:.4f}, AUC: {auc[i]:.4f}, Composite: {comp[i]:.4f}")
print("\n평균 Composite Score:", comp.mean())

# 전체 train_val로 학습 후 test로 최종 성능 평가
pipe.fit(X_trainval, y_trainval)
y_pred = pipe.predict(X_test)
y_prob = pipe.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
comp = (acc + f1 + auc) / 3

print("\n최종 Holdout Test 성능")
print(f"Accuracy : {acc:.4f}")
print(f"F1 Score : {f1:.4f}")
print(f"ROC AUC  : {auc:.4f}")
print(f"Composite: {comp:.4f}")

5-Fold CV (only on train_val)
[Fold 1] Accuracy: 0.6002, F1: 0.4968, AUC: 0.6707, Composite: 0.5892
[Fold 2] Accuracy: 0.6025, F1: 0.4964, AUC: 0.6562, Composite: 0.5850
[Fold 3] Accuracy: 0.5999, F1: 0.5054, AUC: 0.6564, Composite: 0.5873
[Fold 4] Accuracy: 0.5861, F1: 0.4600, AUC: 0.6563, Composite: 0.5675
[Fold 5] Accuracy: 0.5859, F1: 0.4649, AUC: 0.6489, Composite: 0.5666

평균 Composite Score: 0.5791141760983816

최종 Holdout Test 성능
Accuracy : 0.6036
F1 Score : 0.4977
ROC AUC  : 0.6656
Composite: 0.5890
